## Install all dependencies

In [1]:
!pip install PyPDF2
!pip install langchain
!pip install pypdf 
!pip install ollama
!pip install pandas
!pip install openai
!pip install tiktoken
!pip install python-dotenv

Defaulting to user installation because normal site-packages is not writeable
Defaulting to user installation because normal site-packages is not writeable
Defaulting to user installation because normal site-packages is not writeable
Defaulting to user installation because normal site-packages is not writeable
Defaulting to user installation because normal site-packages is not writeable
Defaulting to user installation because normal site-packages is not writeable
Defaulting to user installation because normal site-packages is not writeable
Defaulting to user installation because normal site-packages is not writeable


In [2]:
!pip install --force-reinstall ~/repos/pana/dist/pana-1.0-py3-none-any.whl

Defaulting to user installation because normal site-packages is not writeable
Processing /home/ztkpat001/repos/pana/dist/pana-1.0-py3-none-any.whl
  Attempting uninstall: pana
    Found existing installation: pana 1.0
    Uninstalling pana-1.0:
      Successfully uninstalled pana-1.0


In [3]:
import os
from dotenv import load_dotenv
import json
import inspect
import numpy as np
# import necessary libraries
from pana import ModelFactory, PromptFlow
from pana.texts import nato_text_short, json_nato_flashcards, json_system_message
from pana.lparse import parse_json_string
from langchain_community.document_loaders import PyPDFLoader
import tiktoken

Initializing pana package...


### Models

In [39]:
loader = PyPDFLoader("data/The_New_Invisible_Competitors.pdf")
pages = loader.load_and_split()
pages[0]
# Chunk the content into pieces 
from langchain.text_splitter import RecursiveCharacterTextSplitter
text_splitter = RecursiveCharacterTextSplitter(
    chunk_size=2000, chunk_overlap=0, add_start_index=True
)
all_splits = text_splitter.split_documents(pages)
print("Number of chunks:", len(all_splits))
print(all_splits[1].page_content)

Number of chunks: 16
02/02/2024, 19:48 The New Invisible Competitors | Wilson Quarterly
https://www.wilsonquarterly.com/quarterly/fall-2007-overdrive-competition-in-american-life/the-new-invisible-competitors 2/10Remember the Archie comics? Archie and his conceited rival Reggie battle for
the affections of Betty and Veronica, and the two girls, though they are best
friends, jockey for the attention of Archie, the affable  all- American boy. They
have been at it for more than 60 years, and in the early days the basic situation
wasn’t far removed from the experience of many Americans, especially in
small towns. Indeed, cartoonist Bob Montana based the Archie characters on
people he knew from his high school days during the 1930s in Haverhill,
Massachusetts. Most romantic competition occurred within small groups of
people who knew one another. The girl or guy would choose, perhaps the
couple would marry and settle down, and often the loser ended up living down
the street or across town. R

In [40]:
print(nato_text_short)
print(json_nato_flashcards)
print(json_system_message)


Madam Speaker, Madam Vice President, our First Lady and Second Gentleman. Members of Congress and the Cabinet. Justices of the Supreme Court. My fellow Americans.

Last year COVID-19 kept us apart. This year we are finally together again. 

Tonight, we meet as Democrats Republicans and Independents. But most importantly as Americans.

With a duty to one another to the American people to the Constitution. 

And with an unwavering resolve that freedom will always triumph over tyranny. 

Six days ago, Russia’s Vladimir Putin sought to shake the foundations of the free world thinking he could make it bend to his menacing ways. But he badly miscalculated. 

He thought he could roll into Ukraine and the world would roll over. Instead he met a wall of strength he never imagined.


{"1":{"Question":"How many members are in the NATO Alliance?", "Answer":"30"}, "2":{"Question":"What is the name of the Ukrainian President?", "Answer":"Volodymyr Zelensky"}, "3":{"Question":"Who did the speaker of

In [41]:
json_flow = PromptFlow("json_flow", "Original flow with JSON output now")
json_flow.add_system_message(json_system_message)
print(json_flow)
json_flow.add_interaction("user",
                          f"""
                          {nato_text_short}
                          
                          Make 5 flashcards from the text above.
                          """
                          )
json_flow.add_interaction("assistant", json_nato_flashcards)
json_flow.save_to_file()
json_flow.flow
#json_flow.save_to_file()

[
    {
        "role": "system",
        "content": "\nYou are a helpful study assistant. I want you to create flashcards to be used for studying. The cards should obey these axioms:\n\n        {card_axioms}        \n\n        and have this JSON format:\n\n        {json_card_format}\n\n"
    }
]


[{'role': 'system',
  'content': '\nYou are a helpful study assistant. I want you to create flashcards to be used for studying. The cards should obey these axioms:\n\n        {card_axioms}        \n\n        and have this JSON format:\n\n        {json_card_format}\n\n'},
 {'role': 'user',
  'content': '\n                          \nMadam Speaker, Madam Vice President, our First Lady and Second Gentleman. Members of Congress and the Cabinet. Justices of the Supreme Court. My fellow Americans.\n\nLast year COVID-19 kept us apart. This year we are finally together again. \n\nTonight, we meet as Democrats Republicans and Independents. But most importantly as Americans.\n\nWith a duty to one another to the American people to the Constitution. \n\nAnd with an unwavering resolve that freedom will always triumph over tyranny. \n\nSix days ago, Russia’s Vladimir Putin sought to shake the foundations of the free world thinking he could make it bend to his menacing ways. But he badly miscalculate

In [42]:
i = 2
json_flow_temp = json_flow
json_flow_temp.add_interaction("user", 
f"""
    Based on the following text:

    {all_splits[i].page_content}

    Make 10 flash cards.
"""
)
print(json_flow_temp)

[
    {
        "role": "system",
        "content": "\nYou are a helpful study assistant. I want you to create flashcards to be used for studying. The cards should obey these axioms:\n\n        {card_axioms}        \n\n        and have this JSON format:\n\n        {json_card_format}\n\n"
    },
    {
        "role": "user",
        "content": "\n                          \nMadam Speaker, Madam Vice President, our First Lady and Second Gentleman. Members of Congress and the Cabinet. Justices of the Supreme Court. My fellow Americans.\n\nLast year COVID-19 kept us apart. This year we are finally together again. \n\nTonight, we meet as Democrats Republicans and Independents. But most importantly as Americans.\n\nWith a duty to one another to the American people to the Constitution. \n\nAnd with an unwavering resolve that freedom will always triumph over tyranny. \n\nSix days ago, Russia\u2019s Vladimir Putin sought to shake the foundations of the free world thinking he could make it bend

In [43]:
encoding = tiktoken.encoding_for_model("gpt-3.5-turbo")
pg = all_splits[i].page_content
e = encoding.encode(pg)
encoding.decode(e[0:10])
#print(all_splits[i].page_content[10:100])

'Bangladesh. Hardly anyone saw this challenge coming'

In [44]:
load_dotenv()
OPENAI_API_KEY = os.environ.get("OPENAI_API_KEY")
model = ModelFactory.get_model("gpt-3.5-turbo-0125", api_key=OPENAI_API_KEY)

In [63]:
response = model.make_call(json_flow_temp.flow)

Established connection with gpt-3.5-turbo-0125


In [64]:
print(response)

ChatCompletion(id='chatcmpl-94nu5JP7TPgSfBZu6t7FUksq8TLBg', choices=[Choice(finish_reason='stop', index=0, logprobs=None, message=ChatCompletionMessage(content='\n{"1":{"Question":"Where do some software programmers now come from?", "Answer":"India and Bangladesh"}, "2":{"Question":"What has made it easy to hire specialists from overseas?", "Answer":"Websites like Guru.com"}, "3":{"Question":"What kind of skills were the Bangladeshis learning?", "Answer":"Programming skills and clever ways to sell services online"}, "4":{"Question":"What is one of the examples of services you can hire specialists for overseas?", "Answer":"Designing integrated circuits"}, "5":{"Question":"What kind of workers face new challengers, according to the text?", "Answer":"Factory workers, call center employees, painters, and shovel workers"}, "6":{"Question":"Where can you hire someone to draw up a landscape design for your house?", "Answer":"Websites like Guru.com"}, "7":{"Question":"What is the main theme of

In [65]:
print(response.choices[0].message.content)
cards = response.choices[0].message.content
cards_encoding = encoding.encode(cards)
print("Cards encoding: ", cards_encoding)
print("Length of cards encoding: ", len(cards_encoding))


{"1":{"Question":"Where do some software programmers now come from?", "Answer":"India and Bangladesh"}, "2":{"Question":"What has made it easy to hire specialists from overseas?", "Answer":"Websites like Guru.com"}, "3":{"Question":"What kind of skills were the Bangladeshis learning?", "Answer":"Programming skills and clever ways to sell services online"}, "4":{"Question":"What is one of the examples of services you can hire specialists for overseas?", "Answer":"Designing integrated circuits"}, "5":{"Question":"What kind of workers face new challengers, according to the text?", "Answer":"Factory workers, call center employees, painters, and shovel workers"}, "6":{"Question":"Where can you hire someone to draw up a landscape design for your house?", "Answer":"Websites like Guru.com"}, "7":{"Question":"What is the main theme of the text regarding competition?", "Answer":"Competition is becoming global and accessible through online platforms"}, "8":{"Question":"What country are some eage

In [71]:
a = np.array([280, 316, 288, 284, 309, 302, 284, 271, 285, 321, 291])
print(np.mean(a) / 10)


29.372727272727275


In [28]:
json.loads(response.choices[0].message.content) 

{'1': {'Question': 'Where do many software programmers come from now?',
  'Answer': 'India and Bangladesh'},
 '2': {'Question': 'What kind of skills were Bangladeshis learning?',
  'Answer': 'Programming and clever ways to sell services online'},
 '3': {'Question': 'What are examples of professions facing new challengers due to invisible competition?',
  'Answer': 'Factory workers, call center employees, painters, and shovel workers'},
 '4': {'Question': 'Where can specialists be hired from easily?',
  'Answer': 'Websites like Guru.com'},
 '5': {'Question': 'What variety of services can specialists be hired for overseas?',
  'Answer': 'Designing integrated circuits, composing advertising jingles, painting portraits, drawing landscape designs, etc.'},
 '6': {'Question': 'Which regions from India and Bangladesh are contributing software programmers?',
  'Answer': 'Eager workers'},
 '7': {'Question': 'What is one potential consequence of this invisible competition?',
  'Answer': 'Increase

In [13]:
flashcards = parse_json_string(response.choices[0].message.content)

In [9]:
from openai import OpenAI

In [21]:
def generate_flashcards_batch(msg_chn):
    client = OpenAI(api_key=os.environ.get("OPENAI_API_KEY"))
    response = client.chat.completions.create(
        model="gpt-3.5-turbo-0125",
        messages=msg_chn,
    )
    return response

In [23]:
response = generate_flashcards_batch(json_flow_temp.flow)

In [26]:
print(response.choices[0].message.content)
print(json_flow_temp.flow)


{"1":{"Question":"Where do some software programmers come from now?", "Answer":"India and Bangladesh"}, "2":{"Question":"Who never issued a challenge or threw down a gauntlet?", "Answer":"Bangladeshis"}, "3":{"Question":"What type of workers now face new challengers besides programmers and call center employees?", "Answer":"Factory workers"}, "4":{"Question":"Where can you hire specialists overseas for a wide variety of services?", "Answer":"Guru.com and other websites"}, "5":{"Question":"What can you hire somebody to do through online platforms?", "Answer":"Design integrated circuits or compose advertising jingles"}, "6":{"Question":"What type of design work can someone hire for through online platforms?", "Answer":"Landscape design for a house"}, "7":{"Question":"What is the main idea of the text regarding competition?", "Answer":"Competition is becoming global and touching various professions"}, "8":{"Question":"Where did this unexpected challenge in the job market originate from?"

In [47]:
msg_chn_batches = []
for i in range(4):
    json_flow_temp = PromptFlow.load_from_file("json_flow")
    json_flow_temp.add_interaction("user", 
    f"""
        Based on the following text:

        {all_splits[i].page_content}

        Make 10 flash cards.
    """
    )
    msg_chn_batches.append(json_flow_temp.flow)
    print(json_flow_temp)

[
    {
        "role": "system",
        "content": "\nYou are a helpful study assistant. I want you to create flashcards to be used for studying. The cards should obey these axioms:\n\n        {card_axioms}        \n\n        and have this JSON format:\n\n        {json_card_format}\n\n"
    },
    {
        "role": "user",
        "content": "\n                          \nMadam Speaker, Madam Vice President, our First Lady and Second Gentleman. Members of Congress and the Cabinet. Justices of the Supreme Court. My fellow Americans.\n\nLast year COVID-19 kept us apart. This year we are finally together again. \n\nTonight, we meet as Democrats Republicans and Independents. But most importantly as Americans.\n\nWith a duty to one another to the American people to the Constitution. \n\nAnd with an unwavering resolve that freedom will always triumph over tyranny. \n\nSix days ago, Russia\u2019s Vladimir Putin sought to shake the foundations of the free world thinking he could make it bend

In [48]:
batch_response = generate_flashcards_batch(msg_chn_batches)

[[{'role': 'system',
   'content': '\nYou are a helpful study assistant. I want you to create flashcards to be used for studying. The cards should obey these axioms:\n\n        {card_axioms}        \n\n        and have this JSON format:\n\n        {json_card_format}\n\n'},
  {'role': 'user',
   'content': '\n                          \nMadam Speaker, Madam Vice President, our First Lady and Second Gentleman. Members of Congress and the Cabinet. Justices of the Supreme Court. My fellow Americans.\n\nLast year COVID-19 kept us apart. This year we are finally together again. \n\nTonight, we meet as Democrats Republicans and Independents. But most importantly as Americans.\n\nWith a duty to one another to the American people to the Constitution. \n\nAnd with an unwavering resolve that freedom will always triumph over tyranny. \n\nSix days ago, Russia’s Vladimir Putin sought to shake the foundations of the free world thinking he could make it bend to his menacing ways. But he badly miscalcu